In [ ]:
import os

from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from dotenv import load_dotenv

from azure.ai.agents.models import FilePurpose,FileSearchTool
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent,HostedFileSearchTool,HostedVectorStoreContent

load_dotenv()

In [10]:
async def create_vector_store(client: AIProjectClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = './document.md'
    file = await client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="corporate_insurance")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

In [ ]:
async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        file_id, vector_store = await create_vector_store(client)
        file_search = FileSearchTool(vector_store_ids=[vector_store.vector_store_id])
        
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="PythonRAGAgent",
            instructions="""
                    You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                    - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                    "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                    - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                    - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                    - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                    - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                    You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                    """,
            tools = file_search.definitions,
            tool_resources= file_search.resources
        )
        chat_client=AzureAIAgentClient(agent_id=created_agent.id, async_credential = credential)


        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
            ) as agent:
                    

            print("Agent created. You can now ask questions about the uploaded document.")

            query = "Can you explain Contoso's travel insurance coverage?"

            async for chunk in agent.run_stream(query, tools=HostedFileSearchTool(inputs=vector_store)):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

Uploaded file, file ID: assistant-SZuH66EqoMBfiFwXGZ8i2R
Created vector store, ID: vs_6OQmGwaVc2RzUJIamrk2cuZk
Agent created. You can now ask questions about the uploaded document.
Contoso's travel insurance coverage includes compensation for Bodily Injury, Sickness, or disease sustained by an Insured Person while on a Business Trip, resulting in death or Disablement (Permanent or Temporary, Total or Partial) within 12 months of the incident. Coverage is also provided for loss of or damage to personal goods and business items in the custody of the Insured Person, as well as reimbursement for costs due to cancellation, curtailment, or rearrangement of the Business Trip. Other covered expenses include legal expenses, loss of money and travel documents, and personal liability. Additionally, the policy covers the expenses of search and rescue, evacuation, and repatriation of remains on a worldwide basis. Coverage applies worldwide for any person traveling outside their home country on the 